In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from funs import *

In [2]:
# Locally
# data_url = 'https://github.com/dmika1234/ml_uwr_22/blob/Project/Project/data/fake_job_postings.csv'
data_path = 'data/fake_job_postings.csv'
raw_data = pd.read_csv(data_path)

# For colab
# data_url = '/content/fake_job_postings.csv'
# raw_data = pd.read_csv(data_url, error_bad_lines=False, engine="python")
#straszny problem miałem, żeby wczytać te dane tak ja ty to robiłeś. dziwne błędy mi wyskakiwały

In [3]:
text_colnames = ['company_profile', 'description', 'requirements', 'benefits']
DataPrep = DataPreprocessor()

text_data_ls = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=list)
text_data_np = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=np.array)
text_data_str = DataPrep.preprocess_data(text_data=raw_data, column_names=text_colnames, vectorize_fun=join_fun)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dmika\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
d:\Studia\MachineLearning\ml_uwr_22\Project\funs.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data.fillna('', inplace=True)
d:\Studia\MachineLearning\ml_uwr_22\Project\funs.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

# Word2vec playground

### Our data 1st try

In [21]:
# Train Word2Vec model on job description column
model1 = Word2Vec(text_data_ls['description'], vector_size=20, min_count=1)

In [9]:
model1.wv['growing']

array([-6.4592648 ,  2.1658094 ,  3.445258  ,  6.449147  ,  2.7680726 ,
        8.245474  ,  1.1446196 ,  4.5827475 ,  0.5369002 , -4.02162   ,
        1.5656481 ,  4.4785323 ,  0.6097932 , -3.1391037 ,  2.5798745 ,
       -3.9150026 ,  3.0776334 , -4.092833  , -0.14151247,  0.7096834 ],
      dtype=float32)

In [10]:
model1.wv.most_similar(positive='growing')

[('expanding', 0.8615612983703613),
 ('unbeatable', 0.7613900899887085),
 ('500equinix', 0.7303839325904846),
 ('paced', 0.7217192649841309),
 ('boredommaintaining', 0.7178643941879272),
 ('areseeking', 0.7041919827461243),
 ('experiencedescriptiondo', 0.7030121088027954),
 ('wisebanyan', 0.6996685266494751),
 ('quotaassist', 0.696188747882843),
 ('url_8b2e90799ea4104211c3ad7a24eb3db2a7e5f5a4f6b9a53c4325b4499b1ff797',
  0.6932835578918457)]

In [84]:
def get_agg_word2vec(text, model, vector_size=20, agg_func=np.mean):
    # Get the word2vec representation of each word in the text
    word_vectors = np.array([model.wv[word] for word in text if word in model.wv.index_to_key])
    res = agg_func(word_vectors, axis=0)
    if np.isnan(res).any():
        res = np.zeros(vector_size)
    return res

In [85]:
X_train_vectors = text_data_np['description'].apply(lambda x: get_agg_word2vec(x, model1))

d:\Programy\Anaconda3\envs\UniEnv\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [60]:
X_train_vectors = np.array(X_train_vectors)

True

### Pretrained model on Google news data

https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g

In [ ]:
# # Please note that this code needs only to be run in a fresh runtime.
# # However, it can be rerun afterwards too.
# !pip install -q gdown httpimport
# ![ -e mnist.npz ] || gdown 'https://drive.google.com/uc?id=1QPaC3IKB_5tX6yIZgRgkpcqFrfVqPTXU' -O mnist.npz
!pip install gdown
gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

In [47]:
model_google = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [48]:
model_google.most_similar(positive='growing')

[('grown', 0.6805573105812073),
 ('burgeoning', 0.6775053143501282),
 ('grow', 0.6641595959663391),
 ('rapidly_expanding', 0.6622884273529053),
 ('increasing', 0.6527025699615479),
 ('grows', 0.6311837434768677),
 ('Growing', 0.6133630871772766),
 ('expanding', 0.5987905263900757),
 ('rising', 0.5733980536460876),
 ('growning', 0.5670080780982971)]

In [50]:
len(model_google.index_to_key)

3000000

### Simple example

In [6]:
sentences = [['this', 'is', 'a', 'good', 'example'], ['this', 'is', 'another', 'example']]
model = Word2Vec(sentences, vector_size=10, min_count=1)
model.wv['example']

array([-0.00536227,  0.00236431,  0.0510335 ,  0.09009273, -0.0930295 ,
       -0.07116809,  0.06458873,  0.08972988, -0.05015428, -0.03763372],
      dtype=float32)

In [7]:
model.wv.index_to_key

['example', 'is', 'this', 'another', 'good', 'a']

# TfidVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10)

# Fit the vectorizer to the job description column
X_train_tfidf = vectorizer.fit_transform(text_data_str['description'])